# Imports & Setup
All the imports we needed for processing and writing information to disk.
We used some of the code in assignment 3 and edited things to fit our methods of retrieving.
This file was used on title index, but we used the same file to write text index as well, only changed doc_pair rdd to be with id and text instead of title.

In [1]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [2]:
#import everything
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import math

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# more imports
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [4]:
# get all parquets from bucket
bucket_name = 'ass3_new' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.startswith("multi"):
        paths.append(full_path+b.name)

# Building an inverted index and writing it to disk

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [6]:
# get only parquet files
parquet_paths = [path for path in paths if path.endswith('.parquet')]
parquetFile = spark.read.parquet(*parquet_paths)
# get id and title of each article
doc_text_pairs = parquetFile.select("id", "title").rdd

In [7]:
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [8]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [9]:
from inverted_index_gcp import InvertedIndex

In [6]:
# functions to be used (from assignment 3)

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(id, text):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  # get tokens and regex them
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # stem the tokens
  stemmer = PorterStemmer()
  tokens = [stemmer.stem(t) for t in tokens if t not in all_stopwords]
  # create empty dictionary for mapping
  word_freq = {}
  # iterate over tokens
  for token in tokens:
      if token not in all_stopwords: # if token is not in stopwords
        if token in word_freq: # if token is in dict, increment count
            word_freq[token] += 1
        else:
            word_freq[token] = 1 # if token is not in dict, add it
  # convert dictionary to list of tuples, with term and (doc_id, tf)
  return [(word, (id, freq)) for word, freq in word_freq.items()]

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # sort the list by wiki_id and return it
  return sorted(unsorted_pl, key=lambda x: x[0])

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # calculate df for each token and return result
  return postings.map(lambda x: (x[0], len(x[1])))

# Function to calculate doc length
def doc_len(doc_id, text):
     words = text.split()  # split text into words
     return [(doc_id, len(words))]  # return a tuple of (doc_id, word_count)

# more imports
import pickle
from google.cloud import storage
from pathlib import Path

def write_to_disk(obj, base_dir, name, bucket_name=None):
    '''
    Function to write object to disk.
    
    obj: object being written to disk
    base_dir: base directory
    name: name of pkl file
    bucket_name: the name of the bucket that object is being written to
    '''
    # If writing to a bucket, create the directory if it doesn't exist
    if bucket_name:
        storage_client = storage.Client()
        bucket = storage_client.get_bucket(bucket_name)
        blob = bucket.blob(f"{base_dir}/{name}.pkl")
        blob.upload_from_string(pickle.dumps(obj))
    else:
        path = str(Path(base_dir) / f'{name}.pkl')
        with open(path, 'wb') as f:
            pickle.dump(obj, f)
            
def load_from_bucket(bucket_name, file_path):
    """
    Load a file from a Google Cloud Storage bucket.
    
    Args:
    - bucket_name: Name of the Google Cloud Storage bucket.
    - file_path: Path to the file within the bucket.
    
    Returns:
    - The contents of the file.
    """
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(file_path)
    contents = blob.download_as_string()
    return pickle.loads(contents)

In [ ]:
# Map each row to a key-value pair where id is the key and title is the value
id_title_pairs = doc_text_pairs.map(lambda row: (row['id'], row['title']))

# Collect the key-value pairs into a dictionary
id_title_dict = dict(id_title_pairs.collect())

# write it to bucket
write_to_disk(id_title_dict, "title_index", "id_title_dict", bucket_name)

In [9]:
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# # filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)

This code calculates df dict and len of docs dict, and write them to disk as pkl files

In [2]:
# calculate df dict - dict where {term : num_of_docs}
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# write df to disk
write_to_disk(w2df_dict, "title_index", "df", bucket_name)

# applying word_count function and flattening the result
docs_len = doc_text_pairs.flatMap(lambda x: doc_len(x[0], x[1]))

# collecting the RDD into a dictionary
docs_len_dict = docs_len.collectAsMap()
# write doc_len to disk
write_to_disk(docs_len_dict, "title_index", "docs_len", bucket_name)

calculating idf and writing it

In [ ]:
# get number of documents
N = doc_text_pairs.map(lambda x: x[0]).distinct().count()
# Calculate IDF for each term
idf_rdd = w2df.map(lambda kv: (kv[0], math.log(N / kv[1])))
# Collect IDF values as a dictionary
idf_dict = idf_rdd.collectAsMap()
# write idf to disk
write_to_disk(idf_dict, "title_index", "idf", bucket_name)

We calculated tf-idf for every term-doc pair and stored it in memory as csv files. we did this for titles only.

In [11]:
# get number of documents
N = doc_text_pairs.map(lambda x: x[0]).distinct().count()
# calculate idf
idf = sc.broadcast({term: math.log(N / doc_freq) for term, doc_freq in w2df_dict.items()})
# Calculate TF-IDF for each term-document pair
tf_idf = postings_filtered.flatMap(lambda x: [(x[0], (doc_id, (tf / docs_len_dict[doc_id]) * idf.value[x[0]])) for doc_id, tf in x[1]])
# Convert the TF-IDF RDD to a DataFrame
tf_idf_df = tf_idf.map(lambda x: (x[0], x[1][0], x[1][1])).toDF(["term", "doc_id", "tf_idf"])
# Save the DataFrame as a CSV file
tf_idf_df.repartition(1).write.csv(f'gs://{bucket_name}/title_index/tf_idf_compressed', compression="gzip")

calculate avg doc len and write it as variable

In [11]:
# applying word_count function and flattening the result
docs_len = doc_text_pairs.flatMap(lambda x: doc_len(x[0], x[1]))

# Calculate the total length of all documents
total_length = docs_len.map(lambda length: length[1]).sum()

# Calculate the total number of documents
total_documents = docs_len.count()

# Calculate the average document length
average_length = total_length / total_documents

# write to disk
write_to_disk(average_length, "title_index", "avg_doc_len", bucket_name)

Here we calculated each doc vector for cosine similarity, and saved it in csv file as well.
We first raised every tf in square, then summed all tf^2 that belong to a specific doc, and then took square root
out of them. 

In [15]:
# The result will be (doc_id, tf^2) tuples
doc_tf_pairs = postings_filtered.flatMap(lambda term_postings: [(doc_id, ((tf)**2)) for doc_id, tf in term_postings[1]])
# group by key and raise tf in square
doc_sum_tf = doc_tf_pairs.reduceByKey(lambda tf1, tf2: tf1 + tf2)
# get square root
doc_sqrt_sum_tf = doc_sum_tf.mapValues(lambda sum_tf: math.sqrt(sum_tf))

In [14]:
# Convert RDD to dictionary
doc_sqrt_sum_tf_dict = doc_sqrt_sum_tf.collectAsMap()
# write to disk as pkl
write_to_disk(doc_sqrt_sum_tf_dict, "title_index", "doc_vec_sqr", bucket_name)

In [2]:
# # WRITING AS CSV
# # Convert the TF-IDF RDD to a DataFrame
# doc_sqrt_sum_tf_df = doc_sqrt_sum_tf.map(lambda x: (x[0], x[1])).toDF(["doc_id", "sqr_root"])
# # Save the DataFrame as a CSV file
# doc_sqrt_sum_tf_df.repartition(1).write.csv(f'gs://{bucket_name}/text_index/doc_sqrt_sum_tf_compressed', compression="gzip")

In [ ]:
# load pkls from bucket
doc_vec_sqr = load_from_bucket(bucket_name, "title_index/doc_vec_sqr.pkl")
w2df_dict = load_from_bucket(bucket_name, "title_index/df.pkl")
docs_len_dict = load_from_bucket(bucket_name, "title_index/docs_len.pkl")
avg_doc_len = load_from_bucket(bucket_name, "title_index/avg_doc_len.pkl")
idf = load_from_bucket(bucket_name, "title_index/idf.pkl")

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Add the componenets to index
inverted.df = w2df_dict
inverted.tf_idf = tf_idf_dict
inverted.doc_vector_sqr = doc_vec_sqr
inverted.avg_doc_len = avg_doc_len

# PageRank

This is the code from assignment 3 for generating page rank.

In [1]:
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the
      second entry is the destination page id. No duplicates should be present.
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph
      created by the wikipedia links. No duplicates should be present.
  '''
  # extract source page id and destination page ids from each row
  edges = pages.flatMap(lambda row: [(row.id, link.id) for link in row.anchor_text])
  # remove duplicates from edges RDD
  edges = edges.distinct()
  # extract unique vertices from both source and destination ids
  vertices_source = edges.map(lambda edge: edge[0])
  vertices_dest = edges.map(lambda edge: edge[1])
  # combine source and destination vertices and remove duplicates
  vertices = (vertices_source.union(vertices_dest).distinct()).map(lambda x: (x,))
  return edges, vertices

In [9]:
pages_links = parquetFile.select ("id","anchor_text").rdd
#pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/text_index/page_rank', compression="gzip")

24/03/01 22:04:34 WARN YarnAllocator: Container from a bad node: container_1709327829397_0001_01_000005 on host: cluster-project-w-1.c.assignment3-413720.internal. Exit status: 143. Diagnostics: [2024-03-01 22:04:34.290]Container killed on request. Exit code is 143
[2024-03-01 22:04:34.290]Container exited with a non-zero exit code 143. 
[2024-03-01 22:04:34.290]Killed by external signal
.
24/03/01 22:04:34 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 5 for reason Container from a bad node: container_1709327829397_0001_01_000005 on host: cluster-project-w-1.c.assignment3-413720.internal. Exit status: 143. Diagnostics: [2024-03-01 22:04:34.290]Container killed on request. Exit code is 143
[2024-03-01 22:04:34.290]Container exited with a non-zero exit code 143. 
[2024-03-01 22:04:34.290]Killed by external signal
.
24/03/01 22:04:34 ERROR YarnScheduler: Lost executor 5 on cluster-project-w-1.c.assignment3-413720.internal: Container from a bad node:

In [5]:
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the 
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})

--2024-03-10 23:01:30--  https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.71, 2620:0:861:3:208:80:154:71
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2503235912 (2.3G) [application/octet-stream]
Saving to: ‘pageviews-202108-user.bz2’

pageviews-202108-us 100%[===================>]   2.33G  4.50MB/s    in 9m 5s   

2024-03-10 23:10:36 (4.38 MB/s) - ‘pageviews-202108-user.bz2’ saved [2503235912/2503235912]



NameError: name 'write_to_disk' is not defined

In [7]:
# write dict to disk
write_to_disk(wid2pv, "text_index", "page_views", bucket_name)